# Python for AI Projects

## Introduction

**Generative AI and Agentic AI**

In this Jupyter notebook - we'll quickly setup our Python environment and see how we can run a series of Streamlit dashboards within our Google Colab workspace.

### Executing Code Cells

To execute each cell in this notebook - you can click on the play button on the left of each cell or hit `command/shift + enter` when you're navigating to the cell.

### Setup and Installation

In the following cell we'll run a few commands to install the required Python packages and also grab all of our Streamlit application code and data artefacts from our GitHub repository.

In [ ]:
# ====================
# Initial setup steps
# ====================

# Install Python libraries
!pip install --quiet faiss-cpu==1.11.0
!pip install --quiet ctransformers==0.2.27
!pip install --quiet dotenv==0.9.9

# Clone GitHub repo into a "data" folder
!git clone https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259.git data

# Need to change directory into "data" to download git lfs data objects
%cd data
!git lfs pull

# Then we need to change directory back up so all our paths are correct
%cd ..

# 1. Introduction to Streamlit

Welcome to this hands-on tutorial! In this notebook, we’ll be using Streamlit, a lightweight Python framework for building interactive web dashboards and AI apps — all with minimal code.

## 1.1 What is Streamlit?

Streamlit lets you turn any Python script into a shareable web app using simple commands like st.write(), st.text_input(), and st.button().

It's perfect for:

- Visualizing data and model predictions
- Prototyping dashboards
- Building AI-powered tools quickly

## 1.2 The Challenge in Google Colab

Because Colab runs on remote virtual machines, we can’t directly access localhost ports like we would on our own computers.

That means even if we run:

```python
streamlit run app.py
```

We still won't be able to visit localhost:8501 in our browser to view our application!

## 1.3 Ngrok for Secure Tunnels

To solve this, we’ll use Ngrok, a tool that creates a secure tunnel from the internet to your Colab machine.

We’ll use the following pattern for all of our Streamlit apps in this tutorial:

- Start running our Streamlit apps on port `8051` (we’ll avoid `8501` to reduce potential conflicts)
- Use Ngrok to expose port 8051 to a public web URL
- Visit the generated public URL to view our live dashboards

# 2. Pre-flight Checks

We will need to setup a few secure access components before we can proceed with our Streamlit and AI agents tutorial.

## 2.1 Setup Ngrok Account

1. Create Ngrok account at https://dashboard.ngrok.com/signup
2. Acquire your `AuthToken` at https://dashboard.ngrok.com/get-started/your-authtoken

![ngrok-copy-auth-token](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/ngrok-copy-auth-token.png)

## 2.2 Setup OpenRouter Account

1. Create OpenRouter account at https://openrouter.ai/sign-up
2. Create an API token at https://openrouter.ai/settings/keys with **read only** access and spend limit set to $0

![open-router-create-api-key](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/open-router-create-api-key.png)

3. Store this API key safely as you won't be able to see it again once you click away

> **WARNING 🚨🚨🚨** don't be like me and share your key publicly like in this image! This example is just for illustration and I've already deleted this set of API keys on my account!!!!!!! 🥵

![open-router-copy-api-key](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/open-router-copy-api-key.png)

## 2.3 Streamlit Password

We'll also secure our Streamlit apps by using a simple authentication password. This ensures that our publicly available URL still has a layer of authentication in-memory so only we can access it.

Simply decide on a simple password for your Streamlit applications - but note that you'll need to type this when we start viewing our dashboards, so I wouldn't make it too long or complicated for a better learning experience!

## 2.4 Setup Authentication

Run the code cell immediately below to populate a local `.env` file with your sensitive information we setup in the previous step!

Make sure you have the following ready so you can paste it in when prompted in the following cell:

* Ngrok Auth Token
* OpenRouter API Key
* Streamlit password (remember to keep it short as we'll need to type it for every Streamlit app we run!)

We’ll ask for these values securely using getpass, and store them in a local `.env` file so they’re not exposed in the notebook.

> **Warning 🚨🚨🚨** Make sure to never share your Auth and API keys and be careful to avoid exposing them to the public especially when commiting files into GitHub! I've also setup a secure cleaup step to make sure we remove the `.env` file at [bottom of this notebook!](#final-steps)

In [ ]:
# =========================
# Secure .env Setup Script
# =========================
import getpass
import os
import dotenv

# Step 1: Check if .env file exists
if os.path.exists(".env"):
    print("✅ .env file already exists!")

    # Load existing environment variables
    dotenv.load_dotenv()

    # Check each required variable individually
    ngrok_token = os.getenv("NGROK_AUTH_TOKEN")
    if not ngrok_token:
        ngrok_token = getpass.getpass("🔐 Paste your NGROK_AUTH_TOKEN: ")
        with open(".env", "a") as f:
            f.write(f'NGROK_AUTH_TOKEN="{ngrok_token}"\n')

    openrouter_key = os.getenv("OPEN_ROUTER_API_KEY")
    if not openrouter_key:
        openrouter_key = getpass.getpass("🔐 Paste your OPEN_ROUTER_API_KEY: ")
        with open(".env", "a") as f:
            f.write(f'OPEN_ROUTER_API_KEY="{openrouter_key}"\n')

    streamlit_password = os.getenv("STREAMLIT_PASSWORD")
    if not streamlit_password:
        password_input = getpass.getpass("🔐 Set your STREAMLIT_PASSWORD (press Enter to use default): ")
        streamlit_password = password_input or "linkedin-learning"
        if password_input == "":
            print(f"ℹ️ No password entered — using default: {streamlit_password}")
        else:
            print("✅ Streamlit password set!")
        with open(".env", "a") as f:
            f.write(f'STREAMLIT_PASSWORD="{streamlit_password}"\n')

    print("✅ All required environment variables are now set!")

# Step 2: If .env does not exist, prompt for everything
else:
    print("⚙️ .env file not found — let's create one now!")

    ngrok_token = getpass.getpass("🔐 Paste your NGROK_AUTH_TOKEN: ")
    openrouter_key = getpass.getpass("🔐 Paste your OPEN_ROUTER_API_KEY: ")
    password_input = getpass.getpass("🔐 Set your STREAMLIT_PASSWORD (press Enter to use default): ")

    streamlit_password = password_input or "linkedin-learning"
    if password_input == "":
        print(f"ℹ️ No password entered — using default: {streamlit_password}")
    else:
        print("✅ Streamlit password set! (hidden)")

    with open(".env", "w") as f:
        f.write(f'NGROK_AUTH_TOKEN="{ngrok_token}"\n')
        f.write(f'OPEN_ROUTER_API_KEY="{openrouter_key}"\n')
        f.write(f'STREAMLIT_PASSWORD="{streamlit_password}"\n')

    print("✅ .env file created securely!")
    
# Finally - load in the newly created environment variables
dotenv.load_dotenv()
print("✅ Latest .env file successfully loaded!")

# 3. Streamlit Hello World App

In the next cell, we’ll demonstrate how to run a basic Streamlit app to help verify everything is working smoothly before we dive deeper into our GenAI Streamlit applications

In [ ]:
# Placeholder for Hello World app

# 4. Pure Python LLM App

In the next cell - we'll run a Streamlit app which details a simple `Retrieval-Augmented-Generation` (RAG) pipeline using pure Python only.

This will help us set the stage before we start applying higher-level abstractions using the `LangChain` and `LangGraph` frameworks to modularize our application.

**Placeholder for Exclalidraw diagram**

In [ ]:
# Placeholder for pure Python app

# 5. LangChain App

In the next cell - we'll run a Streamlit app which extends our pure Python app by replacing certain components using `LangChain`

**Placeholder for Exclalidraw diagram**

In [ ]:
# Placeholder for LangChain app

# 5. LangGraph Agentic AI

In the next cell - we'll run a Streamlit app which further extends upon our static AI workflows using `LangChain` to an autonomous agentic AI framework using `LangGraph`

**Placeholder for Exclalidraw diagram**

In [ ]:
# Placeholder for LangGraph app

In [ ]:
# Cleanup - kill any ngrok tunnels and delete sensitive .env file
import os
import dotenv
from pyngrok import ngrok

if os.path.exists(".env"):
    
    # Load environment variables from .env file
    dotenv.load_dotenv()

    # Check if NGROK_AUTH_TOKEN is set and stop any existing tunnels
    if os.getenv("NGROK_AUTH_TOKEN"):
        ngrok.set_auth_token(os.getenv("NGROK_AUTH_TOKEN"))

        # Stop any existing Ngrok tunnels
        ngrok.kill()
        print("✅ Ngrok tunnels stopped.")
    
    # Remove the .env file
    os.remove(".env")
    print("✅ .env file removed safely.")